# Biterm Topic Model

Biterm Topic Model (BTM) es un modelo de tópicos basado en la co-ocurrencia de palabras que aprende los tópicos al modelar patrones de co-ocurrencia palabra-palabra (es decir, biterms). En constraste, LDA y PLSA son modelos de tópicos basados en co-ocurrencias de palabra-documento.

Un _biterm_ consiste en dos palabras co-ocurriendo en el mismo contexto, por ejemplo, en la misma ventana de texto. En contraste con los modelos LDA, BTM modela las ocurrencias de biterms en un corpus. En el proceso de generación, un biterm es generado al extraer dos pañabras independientes del mismo tópico. En otras palabras, la distribución de un biterm $b=(wi, wj)$ se define como:

$$P(b) = \sum_k{P(w_i \:\vert\: z) \cdot P(w_j \:\vert\: z) \cdot P(z)}$$

Con el algoritmo de muestro de Gibbs, podemos extraer los tópicos al estimar $P(w \:\vert\: k)$ y $P(z)$.

# Implementación de BTM (prueba inicial)

A continuación procederemos a ejecutar el código de una implementación de BTM desarrollada por [xiaohuiyan](https://github.com/xiaohuiyan/). El código fuente se puede encontrar en su [repositorio](https://github.com/xiaohuiyan/BTM).

La estructura del proyecto es la siguiente:

```
BTM
│   README.md   
│
└───output
│   │
│   └───model
│
└───sample-data
│   │   doc_info.txt
│   
└───script
│   │   indexDocs.py
│   │   runExample.sh
│   │   topicDisplay.py
│
└───src
    │   ...core de la implementación en C...
```

Para probar esta implementación usaremos la data de ejemplo entregada en el directorio `sample-data`. De ahora en adelante se hablará de documentos correpondiendo a cada línea del archivo `doc_info.txt`. Se puede notar que cada documento tiene una extensión pequeña (como un tweet).

El primer paso es **indexar las palabras en los documentos**. Esto es, mapear cada palabra de los documentos a un ID único comenzando desde el 0.

In [1]:
%run ./BTM/script/indexDocs.py ./BTM/sample-data/doc_info.txt ./BTM/output/doc_wids.txt ./BTM/output/voca.txt

index file: ./BTM/sample-data/doc_info.txt
write file: ./BTM/output/doc_wids.txt
n(w)=28634
write:./BTM/output/voca.txt


Posterior a la ejecución de este script, obtendremos dos archivos.

`doc_wids.txt`: contiene los documentos traducidos con cada palabra reemplazada por el ID asignado.

```
0 1 2 3 4
5 6
7 8 9 10 11 12 9 7 8 9 10 11 13
14 15 16 17 17
18 19 20
21 22 23 24
25 26 27 28
29 30 31
32 33 34 35 36 37 38 39 40 41
42 43 44 45
46 47 48 49
50 51
52 53
```

`voca.txt`: contiene el vocabulario generado posterior al indexado. En otras palabras, contiene los pares `ID  palabra`.

```
0	ipod
1	tumblr
2	app
3	working
4	fine
5	yup
6	longer
7	jungle
8	abduction
9	alive
```

El segundo paso corresponde a **topic learning**. Este paso corresponde a entrenar el modelo usando los documentos representados por los IDs de las palabras.

In [2]:
%%bash
# number of topics
K=20
# alpha, beta
alpha=`echo "scale=3;50/$K"|bc`
beta=0.005
# number of iterations
niter=5
save_step=501
# vocabulary size
W=`wc -l < ./BTM/output/voca.txt`
make -C ./BTM/src
./BTM/src/btm est $K $W $alpha $beta $niter $save_step ./BTM/output/doc_wids.txt ./BTM/output/model/

make: Nothing to be done for `all'.
Run BTM, K=20, W=28634, alpha=2.5, beta=0.005, n_iter=5, save_step=501 ====
load docs: ./BTM/output/doc_wids.txt
Begin iteration
iter 5/5
write p(z): ./BTM/output/model/k20.pz
write p(w|z): ./BTM/output/model/k20.pw_z
cost 3.000405s


Los resultados serán escritos en el directorio `output/model/`:

`k20.pw_z`: una matriz de K*M para P(w|z), con K=20

`k20.pz`: una matriz de K*1 para P(z), con K=20

El tercer paso es **inferir las proporciones de tópicos para cada uno de los documentos**, es decir, P(z|d).

In [3]:
%%bash
K=20
./BTM/src/btm inf sum_b $K ./BTM/output/doc_wids.txt ./BTM/output/model/

Run inference:K=20, type sum_b ====
load p(z):./BTM/output/model/k20.pz
load p(w|z):./BTM/output/model/k20.pw_z
n(z)=20, n(w)=28634
Infer p(z|d) for docs in: ./BTM/output/doc_wids.txt
write p(z|d): ./BTM/output/model/k20.pz_d


El resultado será escrito en el directorio `output/model/`:

`k20.pz_d`: una matriz de N*K para P(z|d), con K=20

El cuarto, y último paso, será **presentar las palabras más frecuentes por cada tópico y sus proporciones en la colección**.

In [4]:
%%bash
K=20
python3 ./BTM/script/topicDisplay.py ./BTM/output/model/ $K ./BTM/output/voca.txt

K:20, n(W):28634
p(z)		Top words
0.056884	0:0.017529 c:0.006421 wind:0.006228 rain:0.004687 00:0.004259 sunday:0.004045 humidity:0.003382 30:0.003339 km:0.003211 12:0.003146
0.053463	lil:0.003370 social:0.002801 makes:0.002573 11:0.002573 fair:0.002482 nigga:0.002436 girls:0.002368 party:0.002345 l:0.002322 years:0.002300
0.052334	cuz:0.003628 boxes:0.003372 wit:0.002954 online:0.002954 bedroom:0.002512 person:0.002489 dnt:0.002442 em:0.002419 bitch:0.002396 hair:0.002349
0.052332	0:0.004326 left:0.003908 c:0.003815 state:0.003210 20:0.003187 22:0.002884 30:0.002745 run:0.002652 pm:0.002559 basketball:0.002349
0.051819	party:0.004792 apple:0.003194 store:0.003171 online:0.002866 iphone:0.002819 app:0.002584 ready:0.002537 hit:0.002396 head:0.002067 concert:0.002067
0.051702	award:0.005014 nominate:0.003296 shorty:0.003272 eat:0.002943 360:0.002754 food:0.002590 hours:0.002566 photo:0.002519 loss:0.002472 dinner:0.002425
0.051340	justin:0.004931 hair:0.003580 die:0.003532 ik:0.003058 ee

Notamos como varios de los tópicos presentados presentan un set de palabras adecuadas que permiten inferir de manera directa el tópico encontrado. Otros tópicos no son muy precisos, por lo que no se puede inferir un tópico claro del set asociado.

# Usando BTM con dataset de tweets

A continuación, procederemos a ejecutar la misma secuencia de scripts ahora para los datasets de tweets parseados en los notebooks anteriores.

## 1. Indexar las palabras en los documentos

In [5]:
%%bash

input_dir=./BTM/sample-data/
output_dir=./BTM/output-tweets/
model_dir=${output_dir}model/
mkdir -p $output_dir/model 

# the input docs for training
doc_pt=${input_dir}tweets-train.txt

echo "=============== Index Docs ============="
# docs after indexing
dwid_pt=${output_dir}doc_wids.txt
# vocabulary file
voca_pt=${output_dir}voca.txt
python3 ./BTM/script/indexDocs.py $doc_pt $dwid_pt $voca_pt

=============== Index Docs =============
index file: ./BTM/sample-data/tweets-train.txt
write file: ./BTM/output-tweets/doc_wids.txt
n(w)=47857
write:./BTM/output-tweets/voca.txt


## 2. Topic Learning

In [6]:
%%bash

# number of topics
K=20

alpha=`echo "scale=3;50/$K"|bc`
beta=0.005
niter=5
save_step=501

input_dir=./BTM/sample-data/
output_dir=./BTM/output-tweets/
model_dir=${output_dir}model/
# the input docs for training
doc_pt=${input_dir}tweets-train.txt
# docs after indexing
dwid_pt=${output_dir}doc_wids.txt
# vocabulary file
voca_pt=${output_dir}voca.txt

## learning parameters p(z) and p(w|z)
echo "=============== Topic Learning ============="
W=`wc -l < $voca_pt` # vocabulary size
make -C ./BTM/src
echo "./BTM/src/btm est $K $W $alpha $beta $niter $save_step $dwid_pt $model_dir"
./BTM/src/btm est $K $W $alpha $beta $niter $save_step $dwid_pt $model_dir

=============== Topic Learning =============
make: Nothing to be done for `all'.
./BTM/src/btm est 20    47857 2.500 0.005 5 501 ./BTM/output-tweets/doc_wids.txt ./BTM/output-tweets/model/
Run BTM, K=20, W=47857, alpha=2.5, beta=0.005, n_iter=5, save_step=501 ====
load docs: ./BTM/output-tweets/doc_wids.txt
Begin iteration
iter 5/5
write p(z): ./BTM/output-tweets/model/k20.pz
write p(w|z): ./BTM/output-tweets/model/k20.pw_z
cost 7.670864s


## 3. Inferir las proporciones de tópicos para cada uno de los documentos

In [7]:
%%bash

# number of topics
K=20

output_dir=./BTM/output-tweets/
# docs after indexing
dwid_pt=${output_dir}doc_wids.txt
model_dir=${output_dir}model/

## infer p(z|d) for each doc
echo "================ Infer P(z|d)==============="
echo "./BTM/src/btm inf sum_b $K $dwid_pt $model_dir"
./BTM/src/btm inf sum_b $K $dwid_pt $model_dir

================ Infer P(z|d)===============
./BTM/src/btm inf sum_b 20 ./BTM/output-tweets/doc_wids.txt ./BTM/output-tweets/model/
Run inference:K=20, type sum_b ====
load p(z):./BTM/output-tweets/model/k20.pz
load p(w|z):./BTM/output-tweets/model/k20.pw_z
n(z)=20, n(w)=47857
Infer p(z|d) for docs in: ./BTM/output-tweets/doc_wids.txt
write p(z|d): ./BTM/output-tweets/model/k20.pz_d


## 4. Presentar las palabras más frecuentes por cada tópico y sus proporciones en la colección

In [8]:
%%bash

K=20   # number of topics

output_dir=./BTM/output-tweets/
model_dir=${output_dir}model/

# vocabulary file
voca_pt=${output_dir}voca.txt

## output top words of each topic
echo "================ Topic Display ============="
python3 ./BTM/script/topicDisplay.py $model_dir $K $voca_pt


================ Topic Display =============
K:20, n(W):47857
p(z)		Top words
0.052902	rajoy:0.009470 pp:0.005375 gobierno:0.004940 mañana:0.004247 españa:0.004202 años:0.003882 @telediariointer:0.003740 2030:0.003722 reforma:0.003625 gracias:0.003065
0.051914	rajoy:0.009098 psoe:0.005658 @telediariointer:0.004735 españa:0.004409 2030:0.003974 chacon:0.003938 rubalcaba:0.003847 eta:0.003838 gobierno:0.003730 madrid:0.003368
0.051597	rajoy:0.007551 gobierno:0.006239 psoe:0.004955 pp:0.004527 congreso:0.003780 mañana:0.003434 gracias:0.003233 rubalcaba:0.003042 madrid:0.002687 reforma:0.002669
0.051456	gobierno:0.007718 rajoy:0.007571 pp:0.004941 españa:0.004521 madrid:0.003854 psoe:0.003681 años:0.003489 millones:0.003169 mañana:0.002886 3:0.002685
0.051253	rajoy:0.006474 psoe:0.006446 rubalcaba:0.004722 congreso:0.004649 @mariviromero:0.004521 gracias:0.004475 gobierno:0.004392 mañana:0.004154 pp:0.004108 chacon:0.003108
0.050823	rajoy:0.008877 pp:0.006389 gobierno:0.005992 psoe:0.0043